In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics
import warnings
import csv
from os.path import exists
from tqdm import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import jpype.imports
import time 



resdir = '/ocean/projects/mth230012p/jleiner/hrt-factor/results/'
datadir = '/ocean/projects/mth230012p/jleiner/hrt-factor/data/'
modeldir = '/ocean/projects/mth230012p/jleiner/hrt-factor/models/'
figdir = 'figures/'

genes = pd.read_csv(datadir + 'missing_gene_alteration_matrix.csv',index_col=0)
metastases = pd.read_csv(datadir + 'metastasis_bysamples.csv',index_col=0)
ds = genes.merge(metastases,on="uid",how="left")
ds.to_pickle(datadir + 'merged_dataset.pkl')
nomissing = ds.dropna(axis='columns')
nomissing = nomissing.iloc[:,:331]
nomissing.columns
nonzeros = nomissing.sum(0) > 400
nonzeros = nonzeros.reset_index()
gene_names = nonzeros[nonzeros[0]]['index'].reset_index(drop=True)
tumor_names = ['adrenal_gland', 'biliary_tract', 'bladder_or_urinary_tract', 'bone',
       'bowel', 'breast', 'cns_brain', 'dist_lymph', 'genital_female',
       'genital_male', 'head_and_neck', 'kidney', 'liver', 'lung', 'lymph',
       'mediastinum', 'other', 'ovary', 'peripheral_nervous_system',
       'peritoneum', 'pleura', 'regional_lymph', 'skin']

In [ ]:

def compile_results(search,duration,label):
    h=search.get_pcalg()[X.shape[1]:].copy(deep=True)
    h.index =  ["Y" + str(j) for j in range(0,Y.shape[1])]
    h=pd.melt(h[h.columns[:X.shape[1]]],ignore_index=False).reset_index()
    h.columns = ["Y_target","X_target","edge"]
    h["Y_target"] = pd.to_numeric(h["Y_target"].str.slice(1))
    h["X_target"] =  pd.to_numeric(h["X_target"].str.slice(1))
    h['is_edge'] = h.apply(lambda row : row["X_target"] in truth[row["Y_target"]],axis=1)
    h['method'] = label
    h['dataset'] = ds
    h['duration'] = duration
    return(h)
   

def get_truth(row): 
    if("collide" in row['Dataset']):
        ds_true = datadir + row['Dataset'] + "_xtrue.npy"
    elif('actualsmall' in row['Dataset'] ):
        return None
    else:
        ds_true = datadir + row['Dataset'] + "_ytrue.npy"
    y_true = np.load(ds_true)
    return row["X_target"] in y_true[row["Y_target"]]


def get_metrics(results,search_type,alpha_list = np.arange(0,1.00,0.005)):
    pvals_comp = results[(results['Type'] == search_type)]
    tpr = [np.sum((pvals_comp['p'] <= alpha)&(pvals_comp['is_edge'] ==1)) / np.sum(pvals_comp['is_edge'] ==1) for alpha in alpha_list] 
    fpr = [ np.sum((pvals_comp['p'] <= alpha)&(pvals_comp['is_edge'] ==0)) / np.sum(pvals_comp['is_edge'] ==0) for alpha in alpha_list]
    precision = [np.sum((pvals_comp['p'] <= alpha)&(pvals_comp['is_edge'] ==1)) / np.sum(pvals_comp['p'] <= alpha) for alpha in alpha_list]
    fdr_bh = [np.sum((pvals_comp['p'] < alpha *( pvals_comp['p'].rank() / pvals_comp.shape[0])) & (~pvals_comp['is_edge'])) / np.sum((pvals_comp['p'] < alpha *( pvals_comp['p'].rank() / pvals_comp.shape[0]))) for alpha in alpha_list]
    power_bh = [np.sum((pvals_comp['p'] < alpha *( pvals_comp['p'].rank() / pvals_comp.shape[0])) & (pvals_comp['is_edge']))/ np.sum(pvals_comp['is_edge']) for alpha in alpha_list]
    return alpha_list, tpr, fpr, fdr_bh, power_bh

def rollup_ds(label,model_type,alpha = 0.2,early_stop = 0.5):
    if(label in ['synth_med','synth']):
        truth = np.load(datadir+ label + "_ytrue.npy")
    elif('semisynth' in label):
        truth = np.load(datadir+ label + "_ytrue.npy")
    elif("collide" in label):
        truth = np.load(datadir+ label + "_xtrue.npy")

    x_dim = np.load(datadir+ label + "_x.npy").shape[1]
    y_dim = np.load(datadir+ label + "_y.npy").shape[1]

    i=0
    outfile = "bash/redo.txt"
    for X_target in trange(x_dim):
        for Y_target in range(y_dim):
            filename = resdir + label.replace('_', '') + "_" + model_type + "_" + str(X_target) + "_" + str(Y_target) + "_combined.pkl"
            if(os.path.exists(filename) != True):
                filename = resdir + label.replace('_', '') + "_" + model_type + "_univariate_" + str(X_target) + "_" + str(Y_target) + "_combined.pkl"
            if(os.path.exists(filename)):
                ds = pd.read_pickle(filename)
                ds.loc[ds['p'] > early_stop,'p'] = 1.0
                if("actualsmall" not in label):
                    ds['is_edge'] = X_target in truth[Y_target]
                ds["X_target"] = X_target
                ds["Y_target"] = Y_target
                if(i==0):
                    ds_comb =ds
                else:
                    ds_comb =pd.concat([ds_comb, ds], ignore_index=True, axis=0)
                i=i+1
            else:
                with open(outfile, 'a') as f:
                    writer = csv.writer(f, delimiter =' ')
                    writer.writerow([label, X_target, Y_target, model_type])

    ds_copy = ds_comb.loc[ds_comb["Type"].isnull()].copy()
    ds_copy.loc[ds_copy["Type"].isnull(),"Type"] = "GSO"
    pd.concat([ds_comb, ds], ignore_index=True, axis=0)
    ds_comb.loc[ds_comb["Type"].isnull(),"Type"] = "Hybrid"
    if("actualsmall" not in label):
        ds_rollup = ds_comb[["X_target","Y_target","Type","is_edge","p"]].groupby(["X_target","Y_target","Type","is_edge"]).max().reset_index()
    else:
        ds_rollup = ds_comb[["X_target","Y_target","Type","p"]].groupby(["X_target","Y_target","Type"]).max().reset_index()
    ds_rollup['gene'] = ds_rollup['X_target'].apply( lambda x : gene_names[x])
    ds_rollup['tumor'] = ds_rollup['Y_target'].apply( lambda x : tumor_names[x])
    ds_rollup.reset_index(drop=True,inplace=True)
    cutoff_FDR = alpha*ds_rollup['p'].rank(method="first")/ds_rollup.shape[0]
    ds_rollup['reject'] = ds_rollup['p'] < cutoff_FDR
    ds_dur = ds_comb[["X_target","Y_target","Type","duration"]].groupby(["X_target","Y_target","Type"]).sum().reset_index()
    ds_rollup = ds_rollup.merge(ds_dur,on=["X_target","Y_target",'Type'])
    return ds_comb, ds_rollup

# Compile Experimental Results 

In [ ]:
ds_list = ['collidesynth-10-10', 'collidesynth-15-15', 'collidesynth-20-20',
           'collidesynth-5-5', 'semisynth-10-10', 'semisynth-15-15','semisynth-20-20',
           'semisynth-5-5','synth_med','synth','synth_collide_large_0.2',
           'synth_collide_large_0.4','synth_collide_large_0.6','synth_collide_large_0.8' ]

i=0
for ds in ds_list:
    for model_type in ['logit','mlp']:
        comb, rollup = rollup_ds(ds,model_type)
        thresh, tpr, fpr, fdr_bh, power_bh = get_metrics(rollup,"Hybrid")
        result_comp = pd.DataFrame(thresh,columns=['thresh'])
        result_comp['dataset'] = ds
        result_comp['model_type'] = model_type
        result_comp['tpr'] = tpr
        result_comp['fpr'] = fpr
        result_comp['fdr_bh'] = fdr_bh
        result_comp['power_bh'] = power_bh
        if "-" in ds: 
            num_S1 = ds.split('-')[1]
            num_S2 = ds.split('-')[2]
            result_comp["num_X"] = num_S1
            result_comp["num_Y"] = num_S2
        if "collide" in ds: 
            result_comp["ds_type"] = "collide"
            if "_" in ds:
                result_comp["corr"] = float(ds.split("_")[-1])

        else:
            result_comp["ds_type"] = "synth"
        if(i==0):
            result_agg = result_comp
        else:
            result_agg = pd.concat([result_agg, result_comp], ignore_index=True, axis=0)
        i=i+1
result_agg.to_pickle(resdir + "scsl_combined.pkl")


ds_list = ['synth_collide_cont_0.1',
           'synth_collide_cont_0.3','synth_collide_cont_0.5','synth_collide_cont_0.7','synth_collide_cont_0.9']

i=0
for ds in ds_list:
    for model_type in ['mlp']:
        comb, rollup = rollup_ds(ds,model_type)
        thresh, tpr, fpr, fdr_bh, power_bh = get_metrics(rollup,"Hybrid")
        result_comp = pd.DataFrame(thresh,columns=['thresh'])
        result_comp['dataset'] = ds
        result_comp['model_type'] = model_type
        result_comp['tpr'] = tpr
        result_comp['fpr'] = fpr
        result_comp['fdr_bh'] = fdr_bh
        result_comp['power_bh'] = power_bh
        if "-" in ds: 
            num_S1 = ds.split('-')[1]
            num_S2 = ds.split('-')[2]
            result_comp["num_X"] = num_S1
            result_comp["num_Y"] = num_S2
        if "collide" in ds: 
            result_comp["ds_type"] = "collide"
            if "_" in ds:
                result_comp["corr"] = float(ds.split("_")[-1])

        else:
            result_comp["ds_type"] = "synth"
        if(i==0):
            result_agg = result_comp
        else:
            result_agg = pd.concat([result_agg, result_comp], ignore_index=True, axis=0)
        i=i+1
result_agg.to_pickle(resdir + "scsl_cont.pkl")


# assign directory
i=0
for filename in os.listdir(resdir):
    f = os.path.join(resdir, filename)
    if (('pcalgo' in f) & ('benchmark' not in f) &('pcalgo_combined' not in f)):
        print(f)
        alpha = float(filename.split("_")[-2])
        temp = pd.read_pickle(f)
        temp['alpha'] = alpha
        if 'collide' in f:
            temp['ds_type'] = 'collide'
            if "large" in f:
                temp["corr"] = float(temp.loc[0,"Dataset"].split("_")[-1])
        if "semisynth" in f:
            temp['ds_type'] = 'synth'
        if "-" in temp.loc[0,"Dataset"]:
            temp['num_X'] = temp.loc[0,"Dataset"].split("-")[1]
            temp['num_Y'] = temp.loc[0,"Dataset"].split("-")[2]
        if i == 0:
            results = temp
        else:
            results = pd.concat([results, temp], ignore_index=True, axis=0)
        i=i+1

results.sort_values(['Dataset'])
results['truth'] = results.apply(get_truth, axis = 1)

results['numer_tpr'] = results['in_graph']*results['truth']
results['numer_fpr'] = results['in_graph']*(~results['truth'])
results['denom_fpr'] = ~results['truth']


pcalgo_agg = results.groupby(['Dataset','Model Type','alpha','ds_type','num_X','num_Y','corr'],dropna=False).sum()
pcalgo_agg.drop(['X_target','Y_target'],axis=1,inplace=True)
pcalgo_agg['fpr'] = (pcalgo_agg['numer_fpr']/pcalgo_agg['denom_fpr'])
pcalgo_agg['tpr'] = (pcalgo_agg['numer_tpr']/pcalgo_agg['truth'])
pcalgo_agg = pcalgo_agg.reset_index()   
pcalgo_agg['fpr_stderr'] =2*np.sqrt((pcalgo_agg['fpr']*(1-pcalgo_agg['fpr']))/pcalgo_agg['denom_fpr'])
pcalgo_agg['tpr_stderr'] =2*np.sqrt((pcalgo_agg['tpr']*(1-pcalgo_agg['tpr']))/pcalgo_agg['truth'])
pcalgo_agg['num_X'] = pcalgo_agg['num_X'].astype(float)
pcalgo_agg['num_Y'] = pcalgo_agg['num_Y'].astype(float)
pcalgo_agg.to_pickle(resdir + "pcalgo_combined.pkl")

# Recreate  Experimental Figures

In [ ]:
# Figure 4
logit_comb, logit_rollup = rollup_ds('synth_med','logit')
ds_comb = logit_comb
ds_rollup = ds_comb[~ds_comb['p_rank_cum'].isnull()]
ds_iterview = ds_rollup.groupby(["Type","iter","Model Type"]).mean()[['p_rank_cum']].reset_index()
plt.plot(ds_iterview[ds_iterview['Type'] == 'Exhaustive']['iter'],np.log(ds_iterview[ds_iterview['Type'] == 'Exhaustive']['p_rank_cum']),label="Naive")
plt.plot(ds_iterview[ds_iterview['Type'] == 'GSO']['iter'],np.log(ds_iterview[ds_iterview['Type'] == 'GSO']['p_rank_cum']),label="GSO",linestyle="dashed")
plt.plot(ds_iterview[ds_iterview['Type'] == 'Hybrid']['iter'],np.log(ds_iterview[ds_iterview['Type'] == 'Hybrid']['p_rank_cum']),label="Hybrid",linestyle="dotted")
plt.xlabel("Iteration")
plt.ylabel("Log-rank of test statistic")
plt.legend()
plt.xlim([-50, 1000])
plt.savefig("iter.pdf")

In [ ]:
scsl = pd.read_pickle(resdir + "scsl_combined.pkl")
pcalgo = pd.read_pickle(resdir + "pcalgo_combined.pkl")
model_type = "logit"
ds = 'synth_med'

scsl_subset = scsl[(scsl['dataset'] == ds)&(scsl['model_type'] == model_type)]
pc_subset = pcalgo[(pcalgo['Dataset'] == ds)&(pcalgo['Model Type'] == model_type)]
plt.plot(scsl_subset['thresh'],scsl_subset['tpr'],label="SCSL (TPR)",color="blue",linestyle = "-",marker= 'v',markevery=5)
plt.plot(pc_subset['alpha'],pc_subset['tpr'],color="brown",marker='x',label="PC-p (TPR)")
plt.plot(scsl_subset['thresh'],scsl_subset['fpr'],label="SCSL (FPR)",color="blue",linestyle = "dashed",marker= 'v',markevery=5)
plt.plot(pc_subset['alpha'],pc_subset['fpr'],color="brown",marker='x',linestyle = "dashed",label="PC-p (FPR)")
plt.plot(pc_subset['alpha'],pc_subset['alpha'],color='black',linestyle='dashed')
plt.xlim([0,0.3])
plt.legend()
plt.xlabel("P-value threshold")
plt.show()

plt.plot(np.log(scsl_subset['thresh']),scsl_subset['fpr']/scsl_subset['thresh'],label="SCSL",color="blue",linestyle = "dashed",marker= 'v',markevery=5)
plt.plot(np.log(pc_subset['alpha']),pc_subset['fpr']/pc_subset['alpha'],color="brown",marker='x',label="PC-p")
plt.axhline(y= 1,color="black",linestyle="dashed")
plt.xlabel("Log of p-value Threshold")
plt.ylabel("Actual FPR / Target")
plt.xlim([-5,-1])

In [ ]:
ds_list = [
['semisynth-5-5',"Algorithm 2 (5, 5)"],   
['semisynth-10-10',"Algorithm 2 (5, 5)"],   
['semisynth-15-15',"Algorithm 2 (15, 15)"],   
['synth_med',"Algorithm 2 (47, 12)"],
['synth',"Algorithm 2 (47, 23)"],
['synth_collide_large_0.2',"Algorithm 3 (0.2)"],
['synth_collide_large_0.4',"Algorithm 3 (0.4)"],
['synth_collide_large_0.6',"Algorithm 3 (0.6)"],
['synth_collide_large_0.8',"Algorithm 3 (0.8)"]]

for datas in ds_list:
    ds = datas[0]
    label = datas[1]
    res_subset = scsl[(scsl["dataset"]==ds)* (scsl["model_type"] == 'logit')]
    pc_subset = pcalgo[(pcalgo["Dataset"]==ds)*(pcalgo["Model Type"] == 'logit')]
    plt.plot(res_subset['thresh'],res_subset['tpr'],color="blue",label="Proposal (TPR)")
    plt.plot(res_subset['thresh'],res_subset['fpr'],color="blue",label="Proposal (FPR)",linestyle ="dashed")
    plt.plot(pc_subset['alpha'],pc_subset['tpr'],color="red",label="PC w/GCM (TPR)")
    plt.plot(pc_subset['alpha'],pc_subset['fpr'],color="red",linestyle = "dashed",label="PC w/GCM (FPR)")
    plt.plot([0.005,0.5],[0.005,0.5],color='black',linestyle='dashed')
    plt.xlim([0.005,0.3])
    plt.legend()
    plt.title(label)
    plt.show()

# Recreate Benchmarking Figures

In [ ]:

method_list = ['fges',
'boss',
'sp',
'grasp',
'pc',
'fci',
'gfci',
'bfci',
'grasp-fci',
'ccd']


ds_list = ['collidesynth-5-5',
'collidesynth-10-10',
'collidesynth-15-15',
'collidesynth-20-20',
'semisynth-5-5',
'semisynth-10-10',
'semisynth-15-15',
'semisynth-20-20',
'synth_med',
'synth',
'synth_collide_large_0.0',
'synth_collide_large_0.1',
'synth_collide_large_0.2',
'synth_collide_large_0.3',
'synth_collide_large_0.4',
'synth_collide_large_0.5',
'synth_collide_large_0.6',
'synth_collide_large_0.7',
'synth_collide_large_0.8',
'synth_collide_large_0.9',
'synth_collide_cont_0.0',
'synth_collide_cont_0.1',
'synth_collide_cont_0.2',
'synth_collide_cont_0.3',
'synth_collide_cont_0.4',
'synth_collide_cont_0.5',
'synth_collide_cont_0.6',
'synth_collide_cont_0.7',
'synth_collide_cont_0.8',
'synth_collide_cont_0.9']

first = False
for ds in ds_list:
    for method in method_list:
        filepath = resdir + ds +'_'+ method +'_benchmarks.pkl'
        if(os.path.exists(filepath)):
            temp = pd.read_pickle(filepath)
            if first == False:
                results = temp
                first = True
            else:
                results = pd.concat([results,temp],axis=0)
results['tp'] = (results['edge'] != 0)&results['is_edge']
results['fn'] = (results['edge'] == 0)&results['is_edge']
results['tn'] = (results['edge'] == 0)&~results['is_edge']
results['fp'] = (results['edge'] != 0)&~results['is_edge']
results_compiled = results.groupby(by=['dataset','method']).sum()
results_compiled['fpr'] = results_compiled['fp']/(results_compiled['tn']+ results_compiled['fp'])
results_compiled['tpr'] = results_compiled['tp']/(results_compiled['tp']+ results_compiled['fn'])
benchmarks_compiled = results_compiled.reset_index() 

figdir = ''

res_large = res_large.sort_values(by = ['num_nodes'])
label = 'gfci'
metric = 'fpr'
plt.plot(res_large[res_large['method'] == label]['num_nodes'],res_large[res_large['method'] == label][metric],label="GFCI",marker='v')
label = 'fges'
plt.plot(res_large[res_large['method'] == label]['num_nodes'],res_large[res_large['method'] == label][metric],label="FGES",marker='s')
label = 'grasp-fci'
plt.plot(res_large[res_large['method'] == label]['num_nodes'],res_large[res_large['method'] == label][metric],label="GRaSP-FCI",marker='.')
label = 'grasp'
plt.plot(res_large[res_large['method'] == label]['num_nodes'],res_large[res_large['method'] == label][metric],label="GRaSP",marker='o')
label = 'boss'
plt.plot(res_large[res_large['method'] == label]['num_nodes'],res_large[res_large['method'] == label][metric],label="BOSS",marker='^')
label = 'pc'
plt.plot(res_large[res_large['method'] == label]['num_nodes'],res_large[res_large['method'] == label][metric],label="PC",marker='>')
#plt.legend()
plt.xlabel("Number of Nodes")
plt.ylabel("False Positive Rate")
plt.ylim(top=0.1)
plt.savefig(figdir  + metric + "_benchmarks.pdf")